# Transient simulation

The CPU temperature is evaluated from its initial temperature, and CPU power level. 

In [6]:
from cpu.systems import CPUSystem

cpu = CPUSystem.load("data/cpu_ref.json")

In [7]:
from cosapp.drivers import NonLinearSolver, LinearDoE, EulerExplicit
from cosapp.recorders import DataFrameRecorder

doe = cpu.add_driver(LinearDoE("doe"))
doe.add_input_var({"fan.T_air": {"lower": 30.0, "upper": 60.0, "count": 201}})

time_driver = doe.add_child(EulerExplicit())
solver = time_driver.add_child(NonLinearSolver("solver", max_iter=15, factor=1.0))

time_driver.time_interval = (0, 5)
time_driver.dt = 0.05

# Define a simulation scenario
time_driver.set_scenario(
    init={"T_cpu": 10.0},
    values={
        # "fan.T_air": 40.,
        "cpu.usage": "100 if time <20 else 0."
    },
)

rec = doe.add_recorder(DataFrameRecorder(includes=["*"]))

cpu.run_drivers()

In [8]:
from cpu.utils.plot_recorders import plot_recorders

plot_recorders(
    {"theoretical": rec.data}, [[("fan.T_air", "cpu.heat_flow_balance")]], width=1200, height=600
)

    'data': [{'mode': 'markers',
              'name': 'theoretic…